# Demand Forecasting (AI, Data Innovation Team, CJ Express)

## File: Training Model (Cluster Model)

 C.J. Express Group Co.,Ltd. All Rights Reserved.
 
 Author: Kao Panboonyuen, Ph.D.
 
 Year: 2021
 ![](https://github.com/kaocj/demandforecasting_darkside/raw/main/demand_main.PNG)

In [1]:
!rm -rf cjdarkside.py
!wget https://github.com/kaocj/demandforecasting_darkside/raw/main/cjdarkside.py -q --show-progress

cjdarkside.py       100%[===================>]  45.29K  --.-KB/s    in 0.001s  


In [2]:
spark

# Import Library

In [3]:
from cjdarkside import *

# Name your CJ DARKSIDE model

In [4]:
version = '2_CJ_BrightSide'
versionFeaturePL = "PL_Feature_v3"

gs_model = "gs://cj-demand-forecasting-dataset/model/2021-March-v1/"
gs_csv = "gs://cj-demand-forecasting-dataset/csv/"
gs_csv_splite = "gs://cj-demand-forecasting-dataset/csv_split/"

PL_FEATURES_PATH = gs_model+"demand_cjdarkside_pl_features_2021_v"+str(versionFeaturePL)+".plmodel"

PL_Bangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_Bangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_bkk_r_binary_model_2021_v"+str(version)+".plmodel"

PL_NotBangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_NotBangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_r_binary_model_2021_v"+str(version)+".plmodel"

# Data Set (Input)

In [5]:
DATASET_PATH = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-0*"
DATASET_PATH_JUST_SPLIT = gs_csv_splite+'sku_v3_split*'
DATASET_PATH_JUST_ONE = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-00000-4827a120-7099-4e1f-bc07-b8f3352c2f12-c000.csv"
TEST_PATH_JUST_ONE = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-00009-4827a120-7099-4e1f-bc07-b8f3352c2f12-c000.csv"

In [6]:
branch_location = 'gs://cj-demand-forecasting-dataset/_additional/BranchDistrictTH_EN_2.csv'
welfare_location = 'gs://cj-demand-forecasting-dataset/_additional/csv_branch_welfare.csv'
reference_qty = 'gs://cj-demand-forecasting-dataset/_reference2/reference00*'
flag_qty = 'gs://cj-demand-forecasting-dataset/_additional/flag_sku_branch.csv'
price_sens = "gs://cj-demand-forecasting-dataset/ad_data/ps_agg_v2_0*"
tier_cj = "gs://cj-demand-forecasting-dataset/tier/tier00000*"

In [7]:
select_variables_initial = ['cj_darkside_id','SalDate','BranchCode','MaterialCode','avgPriceDis',\
                            'avgPrice','supPrice','label','totalNetSale',\
                            'TotalQtySale','types','ZipCode','ProvinceEN','DistrictEN',\
                            'DayofMonth','Yearday','Month','DayofWeek','Year',\
                            'Quarter','WeekOfYear','MonthQuarter','Branch','Name']

# CREATE TRAIN SET

In [8]:
df = spark.read.format("csv").option("header", "true").load(TEST_PATH_JUST_ONE)
df = clean_basic(df)

print('-----------------------------------')
print('-- welcome to the darkside of cj --')
print('-----------------------------------')

df.cache()
print('Done')

-----------------------------------
-- welcome to the darkside of cj --
-----------------------------------
Done


# Start making Features

In [9]:
# Step1) Create Feature: Location

branch_location_df = spark.read.format("csv").option("header", "true").load(branch_location)
branch_location_df = branch_location_df.withColumnRenamed("BranchCode","BranchCodeLocation")

df = df.join(branch_location_df, df.BranchCode == branch_location_df.BranchCodeLocation)

# Step2) Create Feature: All Play with Date and Welfare

df = df.select("*").withColumn("cj_darkside_id", monotonically_increasing_id())

df = df.withColumnRenamed("Date","SalDate")
df = df.withColumn('DayofMonth', F.dayofmonth(F.col('SalDate')))

df = extract_date(df)

df = select_variable(df, select_variables_initial)

wf = read_file(welfare_location,'csv')
df = transform_welfare_flag(df, wf)
df = transform_welfare_flag_day_2(df)

df = df.withColumnRenamed("types","promotion")

df = df.dropna(how='all')

# Feature: ONE HOT ENCODER

In [10]:
# load features pipeline
pipeline_features = Pipeline.read().load(PL_FEATURES_PATH)
df_transformed = pipeline_features.fit(df).transform(df)

# Main (Feature Engineering)

In [11]:
# Show your previous darkside feature -> Here !!!

In [12]:
final_df = clean_binary_2(df_transformed)
final_df.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, label: int, totalNetSale: double, TotalQtySale: int, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector]

# Feature: High-Q

In [13]:
###########################################################################
############### Create Reference Quality Variable #########################
###########################################################################

reference_qty_df = spark.read.format("csv").option("header", "true").load(reference_qty)
reference_qty_df = reference_qty_df.withColumnRenamed("BranchCode","BranchRefQty").withColumnRenamed("MaterialCode","SKURefQty")

reference_qty_df = reference_qty_df.withColumn("BranchRefQty", reference_qty_df["BranchRefQty"].cast("integer"))
reference_qty_df = reference_qty_df.withColumn("SKURefQty", reference_qty_df["SKURefQty"].cast("integer"))

final_df = final_df.join(reference_qty_df,(final_df.BranchCode == reference_qty_df.BranchRefQty) &\
                         (final_df.MaterialCode == reference_qty_df.SKURefQty) &\
                          (final_df.DayofWeek == reference_qty_df.DoW_Num))

###########################################################################
############### Create High Quality Flag Variable #########################
###########################################################################

flag_qty_df = spark.read.format("csv").option("header", "true").load(flag_qty)
#flag_qty_df = flag_qty_df.filter(flag_qty_df.BranchCode.like('%M%') == False)

flag_qty_df = flag_qty_df.dropna()
flag_qty_df = flag_qty_df.withColumn("BranchCode", flag_qty_df["BranchCode"].cast("integer"))
flag_qty_df = flag_qty_df.withColumn("MaterialCode", flag_qty_df["MaterialCode"].cast("integer"))

l_bc = [cj[0] for cj in flag_qty_df.select('BranchCode').distinct().toLocalIterator()]
l_mc = [cj[0] for cj in flag_qty_df.select('MaterialCode').distinct().toLocalIterator()]

matches = (  (F.col('BranchCode').isin(l_bc)) & (F.col('MaterialCode').isin(l_mc))   )
final_df = final_df.withColumn("HighQualityFlag", when(matches, "1").otherwise("0"))

final_df = clean_binary_4(final_df)

In [14]:
# print('now features have: ', final_df.columns)
# print('now show of data: ', final_df.show(2))
# print('now count of data: ', final_df.count())

In [15]:
count_column_types(final_df)

type  count  \
0  bigint  1       
1  double  13      
2  int     21      
3  string  8       
4  vector  5       

                                                                                                                                                                                                                                                                       names  
0  cj_darkside_id                                                                                                                                                                                                                                                             
1  supPrice | AVG_SumQTY_Dow | promotion_idx | ProvinceEN_idx | totalNetSale | avgPriceDis | MostSalesUnitFreq | BranchCode_idx | SD_SumQTY_Dow | MaterialCode_idx | MED_SumQTY_Dow | DistrictEN_idx | avgPrice                                                               
2  MonthQuarter | Month | DayofMonth | ZipCode | NumWeekSale | WeekOfYear | TotalQtySale | Total_SumQTY_Dow | welfareFlag | welfareFlagDay | DayofWeek | BranchRefQty | DoW_Num | Yearday | MaterialCode | Year | BranchCode | HighQualityFlag | SKURefQty | Quarter | label  
3  DistrictEN | Name | MostSalesUnit | Branch | promotion | DoW | ProvinceEN | SalDate                                                                                                                                                                                        
4  MaterialCode_vec | ProvinceEN_vec | promotion_vec | DistrictEN_vec | BranchCode_vec

In [16]:
# # check missing value
# final_df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in final_df.columns)).show()

# Create 1) Data BKK

In [17]:
final_df.groupBy("ProvinceEN").agg(countDistinct("cj_darkside_id")).show(76)

+--------------------+------------------------------+
|          ProvinceEN|count(DISTINCT cj_darkside_id)|
+--------------------+------------------------------+
|          Ayutthaya |                        179682|
|       Pathum Thani |                        134449|
|       Prachin Buri |                         69594|
|       Samut Prakan |                        316967|
|         Ratchaburi |                        375713|
|  Nakhon Ratchasima |                          2831|
|            Bangkok |                        375011|
|        Suphan Buri |                        298856|
|          Chon Buri |                        426992|
|       Chachoengsao |                         74210|
|           Chumphon |                         10071|
|Prachuap Khiri Khan |                        137322|
|          Ang Thong |                         83992|
|       Nakhon Nayok |                         23633|
|           Lop Buri |                         51350|
|           Chai Nat |      

In [18]:
df_bkk = final_df.filter( (final_df.ProvinceEN.like('%Bangkok%') == True) | (final_df.ProvinceEN.like('%Ratchaburi%') == True)  | (final_df.ProvinceEN.like('%Chon Buri%') == True)\
                         | (final_df.ProvinceEN.like('%Nonthaburi%') == True) | (final_df.ProvinceEN.like('%Nakhon Pathom%') == True)  | (final_df.ProvinceEN.like('%Samut Prakan%') == True)\
                         | (final_df.ProvinceEN.like('%Samut Sakhon%') == True))
#df_bkk.show(2)
df_bkk.cache()
df_bkk.count()

2437007

In [19]:
df_bkk.groupBy("BranchCode").agg(countDistinct("cj_darkside_id")).count()

286

# Create 2) Data NOT BKK

In [20]:
df_not_bkk = final_df.filter( (final_df.ProvinceEN.like('%Bangkok%') == False) & (final_df.ProvinceEN.like('%Ratchaburi%') == False)  & (final_df.ProvinceEN.like('%Chon Buri%') == False)\
                         & (final_df.ProvinceEN.like('%Nonthaburi%') == False) & (final_df.ProvinceEN.like('%Nakhon Pathom%') == False)  & (final_df.ProvinceEN.like('%Samut Prakan%') == False)\
                         & (final_df.ProvinceEN.like('%Samut Sakhon%') == False))
#df_not_bkk.show(2)
df_not_bkk.cache()
df_not_bkk.count()

2203646

In [21]:
df_not_bkk.groupBy("BranchCode").agg(countDistinct("cj_darkside_id")).count()

278

# GBT ML Model (Classifier)

In [22]:
############ ----- LOAD ML MODEL ----- ############

read_plmodelC_BKK = PipelineModel.read().load(PL_Bangkok_MODEL_Classification_PATH)
read_plmodelC_NotBKK = PipelineModel.read().load(PL_NotBangkok_MODEL_Classification_PATH)

print (read_plmodelC_BKK.stages)
print (read_plmodelC_NotBKK.stages)

############ ------------------- ----- ############

#read_model = GBTRegressor.read().load(ML_Bangkok_MODEL_Classification_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsC_BKK = read_plmodelC_BKK.transform(df_bkk)
predictionsC_BKK.cache()

predictionsC_NotBKK = read_plmodelC_NotBKK.transform(df_not_bkk)
predictionsC_NotBKK.cache()

print("Testing Process: Done")

[VectorAssembler_815a39300797, GBTClassificationModel (uid=GBTClassifier_0ff8d932d49a) with 5 trees]
[VectorAssembler_815a39300797, GBTClassificationModel (uid=GBTClassifier_0ff8d932d49a) with 5 trees]
Testing Process: Done


In [23]:
predictionsC = predictionsC_BKK.union(predictionsC_NotBKK)
predictionsC.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, label: int, totalNetSale: double, TotalQtySale: int, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector, BranchRefQty: int, SKURefQty: int, DoW: string, DoW_Num: int, Total_SumQTY_Dow: int, AVG_SumQTY_Dow: double, MED_SumQTY_Dow: double, SD_SumQTY_Dow: double, NumWeekSale: int, MostSalesUnit: string, MostSalesUnitFreq: double, HighQualityFlag: int, features_col: vector, rawPrediction: vector, probabil

# GBT ML Model (Regressor)

In [25]:
############ ----- LOAD ML MODEL ----- ############

read_plmodelR_BKK = PipelineModel.read().load(PL_Bangkok_MODEL_Regression_PATH)
read_plmodelR_NotBKK = PipelineModel.read().load(PL_NotBangkok_MODEL_Regression_PATH)

print (read_plmodelR_BKK.stages)
print (read_plmodelR_NotBKK.stages)

############ ----- DONE ML MODEL ----- ############

predictionsR_BKK = read_plmodelR_BKK.transform(df_bkk)
predictionsR_BKK.cache()

predictionsR_NotBKK = read_plmodelR_NotBKK.transform(df_not_bkk)
predictionsR_NotBKK.cache()

print("Testing Process: Done")

[VectorAssembler_f9a973379d14, GBTRegressionModel (uid=GBTRegressor_07dd895bd702) with 5 trees]
[VectorAssembler_f9a973379d14, GBTRegressionModel (uid=GBTRegressor_07dd895bd702) with 5 trees]
Testing Process: Done


In [26]:
predictionsR = predictionsR_BKK.union(predictionsR_NotBKK)
predictionsR.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, label: int, totalNetSale: double, TotalQtySale: int, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector, BranchRefQty: int, SKURefQty: int, DoW: string, DoW_Num: int, Total_SumQTY_Dow: int, AVG_SumQTY_Dow: double, MED_SumQTY_Dow: double, SD_SumQTY_Dow: double, NumWeekSale: int, MostSalesUnit: string, MostSalesUnitFreq: double, HighQualityFlag: int, features_col: vector, predict_qty_lv2: double]

# FINAL PREDICTION (C + R)

In [27]:
predictionsC.show(2)

+--------------+----------+----------+------------+-----------------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|      avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|Bra

In [28]:
predictionsR.show(2)

+--------------+----------+----------+------------+-----------------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+-------------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|      avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|BranchCode_idx|   BranchCode_vec|Materia

In [29]:
def ansByProb(p, t=0.4):
  if p>=t: return 1
  elif p<t: return 0
  else: 0

predictionsA = predictionsC.join(predictionsR,(predictionsC.SalDate == predictionsR.SalDate)&\
                  (predictionsC.BranchCode == predictionsR.BranchCode)&\
                  (predictionsC.MaterialCode == predictionsR.MaterialCode)).select(predictionsC["*"],predictionsR["predict_qty_lv2"])

split1_udf = udf(lambda value: value[0].item(), FloatType())
split2_udf = udf(lambda value: value[1].item(), FloatType())

predictionsA = predictionsA.withColumn('probability_class0', split1_udf('probability')).withColumn('probability_class1', split2_udf('probability'))

ansByP = F.udf(ansByProb, IntegerType())
predictionsA = predictionsA.withColumn("predict_bi_lv1_with_thresh", ansByP("probability_class1"))

predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1")*col("predict_qty_lv2"))
#predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1_with_thresh")*col("predict_qty_lv2"))

In [30]:
predictionsA.select(['SalDate','BranchCode','MaterialCode','label',\
                    'TotalQtySale','predict_bi_lv1','predict_qty_lv2',\
                     'predict_qty_final']).show(2)

+----------+----------+------------+-----+------------+--------------+-------------------+------------------+
|   SalDate|BranchCode|MaterialCode|label|TotalQtySale|predict_bi_lv1|    predict_qty_lv2| predict_qty_final|
+----------+----------+------------+-----+------------+--------------+-------------------+------------------+
|2020-01-04|         2|    20006248|    1|           2|           1.0| 0.7528727601446266|0.7528727601446266|
|2020-01-11|         2|    20006248|    0|           0|           0.0|0.24088159478567628|               0.0|
+----------+----------+------------+-----+------------+--------------+-------------------+------------------+
only showing top 2 rows



# POST PROCESSING (WITH TIME SERIES)

In [31]:
# predictionsA = post_score_v3(predictionsA)
# predictionsA.show(2)

In [32]:
'Hi'

'Hi'

# Criteria 
![](https://github.com/kaocj/demandforecasting_darkside/raw/main/criteria.PNG)

In [35]:
'Hi'

'Hi'